# Plot temperature response over time

## IRF:
Using forcings from RCMIP models and the impulse response function:
\begin{align*}
\text{IRF}(t)=& 0.885\cdot (\frac{0.587}{4.1}\cdot exp(\frac{-t}{4.1}) + \frac{0.413}{249} \cdot exp(\frac{-t}{249}))\\
\text{IRF}(t)= &  \sum_{i=1}^2\frac{\alpha \cdot c_i}{\tau_i}\cdot exp\big(\frac{-t}{\tau_1}\big) 
\end{align*}
with $\alpha = 0.885$, $c_1=0.587$, $\tau_1=4.1$, $c_2=0.413$ and $\tau_2 = 249$.

IRF.** Hvorfor skriver vi ikke bare $IRF(t) =  \sum_{i=1}^2\alpha_i\cdot exp\big(\frac{-t}{\tau_1}\big)$ 

the estimated ERF$_x$ for some forcing agent $x$ as follows: 

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

The ERFs are taken from models in the RCMIP [https://www.rcmip.org/](https://www.rcmip.org/)

# Data availability:

The data is available on request from [https://gitlab.com/rcmip/rcmip](https://gitlab.com/rcmip/rcmip). 

Please contact: Zebedee Nicholls, email: zebedee.nicholls@climate-energy-college.org

# Code + figures

## Imports:

In [1]:
import xarray as xr
from IPython.display import clear_output
import numpy as np
import os
import re
from pathlib import Path
import pandas as pd
import tqdm
from scmdata import df_append, ScmDataFrame
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
from ar6_ch6_rcmipfigs.constants import BASE_DIR
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, INPUT_DATA_DIR, RESULTS_DIR

PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
PATH_DT_OUTPUT = RESULTS_DIR + '/tables/table_sens_dT_cs.csv'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in


## IRF:

In [3]:

def IRF(t, l=0.885, alpha1=0.587 / 4.1, alpha2=0.413 / 249, tau1=4.1, tau2=249):
    """
    Returns the IRF function for:
    :param t: Time in years
    :param l: climate sensitivity factor
    :param alpha1:
    :param alpha2:
    :param tau1:
    :param tau2:
    :return:
    IRF
    """
    return l * (alpha1 * np.exp(-t / tau1) + alpha2 * np.exp(-t / tau2))

In [4]:

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'

## ERF:
Read ERF from file

### Define variables to look at:

In [5]:
# variables to plot:
variables_erf_comp = [
    'Effective Radiative Forcing|Anthropogenic|CH4',
    'Effective Radiative Forcing|Anthropogenic|Aerosols',
    'Effective Radiative Forcing|Anthropogenic|Tropospheric Ozone',
    'Effective Radiative Forcing|Anthropogenic|F-Gases|HFC',
    'Effective Radiative Forcing|Anthropogenic|Other|BC on Snow']
# total ERFs for anthropogenic and total:
variables_erf_tot = ['Effective Radiative Forcing|Anthropogenic',
                     'Effective Radiative Forcing']
# Scenarios to plot:
scenarios_fl = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp370-lowNTCF-aerchemmip',  # 'ssp370-lowNTCF', Due to mistake here
                'ssp585', 'historical']

## Open dataset:

In [6]:
ds = xr.open_dataset(PATH_DATASET)

# Integrate:
The code below integrates the read in ERFs with the pre defined impulse response function (IRF).

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

In [7]:
name_deltaT = 'Delta T'


def new_varname(var, nname):
    """
    var:str
        Old variable of format varname|bla|bla
    nname:str
        name for the resulting variable, based on var
    Returns
    -------
    new variable name with nname|bla|bla
    """
    return nname + '|' + '|'.join(var.split('|')[1:])


def integrate_(i, var, nvar, ds, ds_DT, csfac=0.885):
    """
    
    Parameters
    ----------
    i:int
        the index for the integral
    var:str
        the name of the EFR variables to integrate
    nvar:str
        the name of output integrated value

    ds:xr.Dataset
        the ds with the intput data
    ds_DT: xr.Dataset
        the ouptut ds with the integrated results
    csfac: climate sensitivity factor (for IRF)
    Returns
    -------
    None
    
    """
    # lets create a ds that goes from 0 to i inclusive
    ds_short = ds[{'time': slice(0, i + 1)}].copy()
    # lets get the current year
    current_year = ds_short['time'][{'time': i}].dt.year
    # lets get a list of years
    years = ds_short['time'].dt.year
    # lets get the year delta until current year(i)
    ds_short['end_year_delta'] = current_year - years

    # lets get the irf values from 0 until i
    ds_short['irf'] = IRF(
        ds_short['end_year_delta'] * ds_short['delta_t'], l=csfac
    )

    # lets do the famous integral
    ds_short['to_integrate'] = \
        ds_short[var] * \
        ds_short['irf'] * \
        ds_short['delta_t']

    # lets sum all the values up until i and set
    # this value at ds_DT
    # If whole array is null, set value to nan
    if np.all(ds_short['to_integrate'].isnull()):  # or last_null:
        _val = np.nan
    else:
        # 

        _ds_int = ds_short['to_integrate'].sum(['time'])
        # mask where last value is null (in order to not get intgral 
        _ds_m1 = ds_short['to_integrate'].isel(time=-1)
        # where no forcing data)
        _val = _ds_int.where(_ds_m1.notnull())
    # set value in dataframe:
    ds_DT[nvar][{'time': i}] = _val


def integrate_to_dT(ds, from_t, to_t, variables, csfac=0.885):
    """
    Integrate forcing to temperature change.

    :param ds: dataset containing the focings
    :param from_t: start time
    :param to_t: end time
    :param variables: variables to integrate
    :param csfac: climate sensitivity factor
    :return:
    """
    # slice dataset
    ds_sl = ds.sel(time=slice(from_t, to_t))
    len_time = len(ds_sl['time'])
    # lets create a result DS
    ds_DT = ds_sl.copy()

    # lets define the vars of the ds
    vars = variables  # variables_erf_comp+ variables_erf_tot #['EFR']
    for var in variables:
        namevar = new_varname(var, name_deltaT)
        # set all values to zero for results dataarray:
        ds_DT[namevar] = ds_DT[var] * 0
        # Units Kelvin:
        ds_DT[namevar].attrs['unit'] = 'K'
        if 'unit' in ds_DT[namevar].coords:
            ds_DT[namevar].coords['unit'] = 'K'

    for i in range(len_time):
        # da = ds[var]
        if (i % 20) == 0:
            print('%s of %s done' % (i, len_time))
        for var in variables:
            namevar = new_varname(var, name_deltaT)  # 'Delta T|' + '|'.join(var.split('|')[1:])

            # print(var)
            integrate_(i, var, namevar, ds_sl, ds_DT, csfac=csfac)
    clear_output()

    fname = 'DT_%s-%s.nc' % (from_t, to_t)
    # save dataset.
    ds_DT.to_netcdf(fname)
    #for var in variables:
    #    ds_DT[var].plot(alpha=0.3)
    # plt.show()
    #plt.show()
    return ds_DT



# Table

## Compute $\Delta T$ with 3 different climate sensitivities

In [4]:
0.884-0.526

0.358

In [3]:
1.136-0.884

0.2519999999999999

In [8]:
csfs = [0.884, 0.526, 1.136]
ECS2ecsf = {'ECS = 2K':0.526, 'ECS = 3.4K':0.884, 'ECS = 5K': 1.136 }
dic_ds = {}
for key  in ECS2ecsf:
    dic_ds[key] = integrate_to_dT(ds, '1850', '2100',(variables_erf_comp+variables_erf_tot), csfac=ECS2ecsf[key])

## Set reference year

In [9]:
ref_year = '2021'

In [10]:
years= ['2040', '2100']
iterables = [list(ECS2ecsf.keys()), years]

def setup_table(scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    table = pd.DataFrame(columns=[var.split('|')[-1] for var in variables_erf_comp], index = _i).transpose()
    table.index.name=scenario_n
    return table



In [12]:
scntab_dic = {}
for scn in scenarios_fl:
    tab = setup_table(scenario_n=scn)
    for var in variables_erf_comp:
        tabvar = var.split('|')[-1]
        dtvar = new_varname(var, name_deltaT)
        for key in ECS2ecsf:
            for year in years: 
                
                _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                #print(_tab_da)

                #_tab_da = dic_ds[key][var].sel(scenario=scn, time=slice(year,year))
                tab.loc[tabvar,key][year]=_tab_da.mean('climatemodel').values[0]
    scntab_dic[scn]=tab.copy()


#tab

/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning:

In [13]:
from IPython.display import display

for key in scntab_dic:
    display(scntab_dic[key])

ECS = 2K              ECS = 3.4K              \
                          2040        2100        2040        2100   
ssp119                                                               
CH4                 -0.0265983   -0.129691  -0.0447013    -0.21796   
Aerosols              0.156464     0.22492    0.262955    0.378002   
Tropospheric Ozone  -0.0499251   -0.100481  -0.0839045   -0.168869   
HFC                 0.00346825 -0.00297617  0.00582878 -0.00500177   
BC on Snow          -0.0113291   -0.016979  -0.0190398  -0.0285351   

                      ECS = 5K              
                          2040        2100  
ssp119                                      
CH4                 -0.0574442   -0.280094  
Aerosols              0.337914    0.485758  
Tropospheric Ozone   -0.107823   -0.217008  
HFC                 0.00749037 -0.00642762  
BC on Snow          -0.0244674  -0.0366695

ECS = 2K             ECS = 3.4K               ECS = 5K  \
                          2040        2100       2040        2100       2040   
ssp126                                                                         
CH4                 -0.0173677   -0.123156 -0.0291883   -0.206977  -0.037509   
Aerosols              0.122156    0.227741   0.205296    0.382743    0.26382   
Tropospheric Ozone  -0.0337634  -0.0940473  -0.056743   -0.158057 -0.0729186   
HFC                  0.0110322 -0.00132423  0.0185408 -0.00222552  0.0238262   
BC on Snow         -0.00943653  -0.0153264 -0.0158591  -0.0257577   -0.02038   

                                
                          2100  
ssp126                          
CH4                  -0.265979  
Aerosols              0.491851  
Tropospheric Ozone   -0.203113  
HFC                -0.00285994  
BC on Snow          -0.0331004

ECS = 2K             ECS = 3.4K              \
                           2040       2100        2040        2100   
ssp245                                                               
CH4                   0.0289374 -0.0041827   0.0486324 -0.00702949   
Aerosols               0.036379   0.136759   0.0611389    0.229839   
Tropospheric Ozone  0.000932831 -0.0390347  0.00156772   -0.065602   
HFC                   0.0182347  0.0384514   0.0306454   0.0646217   
BC on Snow          -0.00340278 -0.0124888 -0.00571875  -0.0209888   

                      ECS = 5K              
                          2040        2100  
ssp245                                      
CH4                   0.062496 -0.00903337  
Aerosols             0.0785676    0.295358  
Tropospheric Ozone  0.00201463   -0.084303  
HFC                  0.0393814   0.0830433  
BC on Snow         -0.00734898   -0.026972

ECS = 2K              ECS = 3.4K               ECS = 5K  \
                          2040        2100        2040       2100        2040   
ssp370                                                                          
CH4                  0.0545716    0.215658   0.0917135   0.362437    0.117858   
Aerosols            -0.0110689 -0.00742148  -0.0186025 -0.0124726  -0.0239054   
Tropospheric Ozone   0.0246028   0.0634972   0.0413476   0.106714   0.0531345   
HFC                  0.0170488   0.0649333   0.0286523   0.109127   0.0368202   
BC on Snow          0.00368423  0.00675406  0.00619175  0.0113509  0.00795682   

                               
                         2100  
ssp370                         
CH4                  0.465756  
Aerosols           -0.0160281  
Tropospheric Ozone   0.137135  
HFC                  0.140236  
BC on Snow          0.0145867

ECS = 2K              ECS = 3.4K             \
                                 2040        2100        2040       2100   
ssp370-lowNTCF-aerchemmip                                                  
CH4                         0.0545716    0.215658   0.0917135   0.362437   
Aerosols                    0.0627329    0.142449    0.105429   0.239401   
Tropospheric Ozone        -0.00320627   0.0120464 -0.00538849  0.0202453   
HFC                         0.0170488   0.0649333   0.0286523   0.109127   
BC on Snow                -0.00374333 -0.00704176 -0.00629107 -0.0118344   

                             ECS = 5K             
                                 2040       2100  
ssp370-lowNTCF-aerchemmip                         
CH4                          0.117858   0.465756  
Aerosols                     0.135484   0.307646  
Tropospheric Ozone        -0.00692457  0.0260165  
HFC                         0.0368202   0.140236  
BC on Snow                -0.00808445 -0.0152081

ECS = 2K              ECS = 3.4K              \
                          2040        2100        2040        2100   
ssp585                                                               
CH4                  0.0441234    0.114892   0.0741541    0.193089   
Aerosols             0.0471011   0.0979238   0.0791586    0.164572   
Tropospheric Ozone   0.0153166   0.0113907   0.0257412   0.0191433   
HFC                  0.0271078    0.149843   0.0455577    0.251827   
BC on Snow         -0.00181596 -0.00353266 -0.00305192 -0.00593702   

                      ECS = 5K              
                          2040        2100  
ssp585                                      
CH4                   0.095293    0.248132  
Aerosols              0.101724    0.211486  
Tropospheric Ozone   0.0330793   0.0246005  
HFC                  0.0585447    0.323615  
BC on Snow         -0.00392193 -0.00762947

ECS = 2K              ECS = 3.4K              \
                           2040        2100        2040        2100   
historical                                                            
CH4                  0.00771324   0.0270393   0.0129629   0.0454424   
Aerosols            -0.00990863  -0.0425141  -0.0166525  -0.0714495   
Tropospheric Ozone   0.00524164   0.0184926  0.00880914   0.0310788   
HFC                 0.000744435  0.00177428   0.0012511  0.00298186   
BC on Snow                  NaN         NaN         NaN         NaN   

                      ECS = 5K              
                          2040        2100  
historical                                  
CH4                  0.0166582   0.0583966  
Aerosols            -0.0213996  -0.0918175  
Tropospheric Ozone   0.0113203   0.0399383  
HFC                 0.00160775  0.00383189  
BC on Snow                 NaN         NaN

In [14]:
years= ['2040', '2100']
iterables = [list(ECS2ecsf.keys()), years]
iterables2 = [scenarios_fl, [var.split('|')[-1] for var in variables_erf_comp]]

def setup_table2():#scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    _r = pd.MultiIndex.from_product(iterables2, names=['', ''])
    
    #table = pd.DataFrame(columns=[var.split('|')[-1] for var in variables_erf_comp], index = _i).transpose()
    table = pd.DataFrame(columns=_r, index = _i).transpose()
    #table.index.name=scenario_n
    return table
#table['ECS = 2K']
#table



In [17]:
#scntab_dic = {}
tab = setup_table2()#scenario_n=scn)

for scn in scenarios_fl:
    for var in variables_erf_comp:
        tabvar = var.split('|')[-1]
        dtvar = new_varname(var,name_deltaT)
        print(dtvar)
        for key in ECS2ecsf:
            for year in years: 
                
                _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                #print(_tab_da)

                #_tab_da = dic_ds[key][var].sel(scenario=scn, time=slice(year,year))
                #print(_tab_da['climatemodel'])
                tab.loc[(scn, tabvar), (key,year)] =_tab_da.mean('climatemodel').values[0]
    #scntab_dic[scn]=tab.copy()


#tab

Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthr

/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning:

In [18]:
tab

ECS = 2K              \
                                                     2040        2100   
                                                                        
ssp119                    CH4                  -0.0265983   -0.129691   
                          Aerosols               0.156464     0.22492   
                          Tropospheric Ozone   -0.0499251   -0.100481   
                          HFC                  0.00346825 -0.00297617   
                          BC on Snow           -0.0113291   -0.016979   
ssp126                    CH4                  -0.0173677   -0.123156   
                          Aerosols               0.122156    0.227741   
                          Tropospheric Ozone   -0.0337634  -0.0940473   
                          HFC                   0.0110322 -0.00132423   
                          BC on Snow          -0.00943653  -0.0153264   
ssp245                    CH4                   0.0289374  -0.0041827   
                          Aerosols               0.036379    0.136759   
                          Tropospheric Ozone  0.000932831  -0.0390347   
                          HFC                   0.0182347   0.0384514   
                          BC on Snow          -0.00340278  -0.0124888   
ssp370                    CH4                   0.0545716    0.215658   
                          Aerosols             -0.0110689 -0.00742148   
                          Tropospheric Ozone    0.0246028   0.0634972   
                          HFC                   0.0170488   0.0649333   
                          BC on Snow           0.00368423  0.00675406   
ssp370-lowNTCF-aerchemmip CH4                   0.0545716    0.215658   
                          Aerosols              0.0627329    0.142449   
                          Tropospheric Ozone  -0.00320627   0.0120464   
                          HFC                   0.0170488   0.0649333   
                          BC on Snow          -0.00374333 -0.00704176   
ssp585                    CH4                   0.0441234    0.114892   
                          Aerosols              0.0471011   0.0979238   
                          Tropospheric Ozone    0.0153166   0.0113907   
                          HFC                   0.0271078    0.149843   
                          BC on Snow          -0.00181596 -0.00353266   
historical                CH4                  0.00771324   0.0270393   
                          Aerosols            -0.00990863  -0.0425141   
                          Tropospheric Ozone   0.00524164   0.0184926   
                          HFC                 0.000744435  0.00177428   
                          BC on Snow                  NaN         NaN   

                                              ECS = 3.4K              \
                                                    2040        2100   
                                                                       
ssp119                    CH4                 -0.0447013    -0.21796   
                          Aerosols              0.262955    0.378002   
                          Tropospheric Ozone  -0.0839045   -0.168869   
                          HFC                 0.00582878 -0.00500177   
                          BC on Snow          -0.0190398  -0.0285351   
ssp126                    CH4                 -0.0291883   -0.206977   
                          Aerosols              0.205296    0.382743   
                          Tropospheric Ozone   -0.056743   -0.158057   
                          HFC                  0.0185408 -0.00222552   
                          BC on Snow          -0.0158591  -0.0257577   
ssp245                    CH4                  0.0486324 -0.00702949   
                          Aerosols             0.0611389    0.229839   
                          Tropospheric Ozone  0.00156772   -0.065602   
                          HFC                  0.0306454   0.0646217   
                          BC on Snow         -0.00

In [19]:
tab.to_csv(PATH_DT_OUTPUT)